# Data Querying and Exporting

This notebook demonstrates the process of querying data from Delta Lake and exporting it in various formats.

## Starting the servers:
To launch the server, open the Docker Desktop app and run the following command at the root of the project:
```bash
$ make up
```
This command will launch the Jupyter server using the environment variables defined in the `.env` file.

#### Understanding expected file paths:
DiveDB expects the following paths to be set in the `.env` file:
- `CONTAINER_DATA_PATH`
- `LOCAL_DATA_PATH`
- `HOST_DELTA_LAKE_PATH`
- `CONTAINER_DELTA_LAKE_PATH`

These paths are used to mount the Delta Lake and file storage to the containers. The "LOCAL_" and "HOST_" paths can be wherever makes sense for your local machine. The "CONTAINER_" paths are the paths that the containers expect. We recommend you keep the "CONTAINER_" paths as they are in the `.env.example` file.

#### When is the server ready?
You'll know it's ready when you see the following logs in the terminal:
```bash
jupyter-1            | [I 2024-08-30 16:12:37.083 ServerApp] Serving notebooks from local directory: /app
jupyter-1            | [I 2024-08-30 16:12:37.083 ServerApp] Jupyter Server 2.14.2 is running at:
jupyter-1            | [I 2024-08-30 16:12:37.083 ServerApp] http://e29d05e13fd0:8888/jupyter/tree
jupyter-1            | [I 2024-08-30 16:12:37.083 ServerApp]     http://127.0.0.1:8888/jupyter/tree
```

## Connecting to the Jupyter Kernel:
To connect to the Jupyter server in your notebook, follow these steps:
1. Click the "Select Kernel" button at the top right of the page.
1. Pick the "Select another kernel" option in the dropdown menu.
1. Pick the "Existing Jupyter Server" option in the dropdown menu.
1. Now we need to connect to the Jupyter server.
    - If you previously connected to the Jupyter server
        - Pick the "localhost" option in the dropdown menu (or whatever you named it prior)
    - If you have not connected to the Jupyter server before
        - Pick the "Enter the URL of the running Jupyter server" option in the dropdown menu.
        - Enter http://localhost:8888/jupyter
        - Give it a name you'll remember (like "Local DiveDB Jupyter Server")
1. Press the "Reload" icon in the top right of the dropdown menu to see the latest kernel.
1. Pick the "Python 3" option in the dropdown menu.

This will ensure you execute the Jupyter notebook in the correct environment.

## Querying from Delta Lake
We connect to our datastores using the `DuckPond` class. DuckPond is a wrapper around a DuckDB connection with access to both our Metadata Database and our measurements stored in Delta Lake. The ability to query both sources of data from a single connection is useful for quickly accessing data for analysis.

There are two main ways to query data from Delta Lake:
1. Using the DuckPond `get_delta_data` method
2. Using the DuckPond connection to query directly

### Using the DuckPond `get_delta_data` method
DuckPond's `get_delta_data` method constructs a query based on the parameters you pass to it and returns a DuckDB DataFrame. It is useful for quickly accessing data for analysis. It takes the following optional parameters:
- `labels`: A string or list of data labels to query.
- `logger_ids`: A string or list of logger IDs to query.
- `animal_ids`: A string or list of animal IDs to query.
- `deployment_ids`: A string or list of deployment IDs to query.
- `recording_ids`: A string or list of recording IDs to query.
- `date_range`: A tuple of start and end dates to query.
- `limit`: The maximum number of rows to return.

The `get_delta_data` method returns a [DuckDB DuckDBPyConnection](https://duckdb.org/docs/api/python/reference/#duckdb.DuckDBPyConnection) which can be used to convert the data in many different formats including the following ([see documentation for a full list](https://duckdb.org/docs/api/python/conversion#result-conversion-duckdb-results-to-python))
- NumPy Array (`.fetchnumpy()`)
- Pandas DataFrame (`.df()`)
- Arrows Table (`.arrow()`)
- Polars DataFrame (`.pl()`)

Until a conversion method is called, the data is not loaded into memory. This allows for large queries to be run without using too much memory.

##### Example:

In [ ]:
import os
import importlib
import DiveDB.services.duck_pond
importlib.reload(DiveDB.services.duck_pond)

from DiveDB.services.duck_pond import DuckPond

duckpond = DuckPond(os.environ["LOCAL_DELTA_PATH"])

conn = duckpond.conn.sql("SELECT count(*) FROM DataLake").df()

display(conn)

## Querying for shared frequency from Delta Lake
Delta Lake can store multiple signal names at a single frequency. If you query a single signal name, the data will be returned as a list of values for each timestamp. If you query multiple signal names, the data will be returned as a list of lists of values for each timestamp.

The data will be returned as a Pandas DataFrame with a DatetimeIndex.

##### Example:

In [ ]:
import os
import importlib
import DiveDB.services.duck_pond
importlib.reload(DiveDB.services.duck_pond)

from DiveDB.services.duck_pond import DuckPond

duckpond = DuckPond(os.environ["LOCAL_DELTA_LAKE"])

duckpond.get_delta_data(    
    labels=["derived_data_depth"],
    animal_ids="apfo-001a",
    frequency=1/60,  # Once a minute
)


### Using the DuckPond connection to query directly
More complex queries can be run directly on the DuckPond connection. This is useful for queries that may not be supported by the `get_delta_data` method which has those involving grouping or aggregations. 

DuckDB runs sql very similar in syntax to other SQL databases. A full breakdown of the syntax can be found [in the documenation](https://duckdb.org/docs/sql/introduction).

The connection object can be found in the `duckpond.conn` attribute. To run queries, use the `sql` method which also returns a [DuckDB DuckDBPyConnection](https://duckdb.org/docs/api/python/reference/#duckdb.DuckDBPyConnection) which can be used to convert the data in many different formats including the following ([see documentation for a full list](https://duckdb.org/docs/api/python/conversion#result-conversion-duckdb-results-to-python))
- NumPy Array (`.fetchnumpy()`)
- Pandas DataFrame (`.df()`)
- Arrows Table (`.arrow()`)
- Polars DataFrame (`.pl()`)

##### Example:

In [ ]:
import importlib
import os
# Reload the DuckPond module to pick up any changes
import DiveDB.services.duck_pond
importlib.reload(DiveDB.services.duck_pond)
from DiveDB.services.duck_pond import DuckPond

duckpond = DuckPond(os.environ["CONTAINER_DELTA_LAKE_PATH"])

df = duckpond.conn.sql(f"""
    SELECT label, avg(value) as mean_data
    FROM (
        SELECT label, value.float as value
        FROM DataLake
        WHERE label = 'sensor_data_light'
        OR label = 'sensor_data_temperature'
    )
    GROUP BY label
""").df()

display(df)


## Chaining Queries
Queries can be chained together to form a pipeline. This is useful for running complex queries that involve multiple steps.

##### Example:

In [ ]:
import importlib
import os
# Reload the DuckPond module to pick up any changes
import DiveDB.services.duck_pond
importlib.reload(DiveDB.services.duck_pond)
from DiveDB.services.duck_pond import DuckPond

duckpond = DuckPond(os.environ["LOCAL_DELTA_PATH"])

# Get the filtered data
filtered_data = duckpond.get_delta_data(    
    animal_ids="apfo-001a",
    # Resample values to 10 Hz and make sure each signal has the same time intervals
    frequency=10,
    # Aggregation of events (think state events - behaviors) type: state (has state and end dates)
    classes="sensor_data_accelerometer",
    
)

display(filtered_data)


## Query Variables
Sometimes we don't want to hardcode variables in our queries. We can use the `execute` method to pass variables to the query.

##### Example:

In [ ]:
import importlib
import os
# Reload the DuckPond module to pick up any changes
import DiveDB.services.duck_pond
importlib.reload(DiveDB.services.duck_pond)
from DiveDB.services.duck_pond import DuckPond

duckpond = DuckPond(os.environ["CONTAINER_DELTA_LAKE_PATH"])

label = "sensor_data_temperature"
df = duckpond.conn.execute(f"""
    SELECT label, avg(value) as mean_data
    FROM (
        SELECT label, value.float as value
        FROM DataLake
        WHERE label = $1
    )
    GROUP BY label
""", [label]).df()

display(df)

## Query Metadata Database
We can also query the Metadata Database directly. This is useful for querying data that is not stored in Delta Lake and joining it for queries on measurement data.

##### Example:

In [ ]:
import importlib
import os
# Reload the DuckPond module to pick up any changes
import DiveDB.services.duck_pond
importlib.reload(DiveDB.services.duck_pond)
from DiveDB.services.duck_pond import DuckPond

duckpond = DuckPond(os.environ["LOCAL_DELTA_LAKE"])


# Show all tables we have access to
print(duckpond.get_db_schema())

df = duckpond.conn.sql("""
    SELECT value.float as value
    FROM DataLake 
    JOIN Metadata.public.Animals ON DataLake.animal = Animals.id
    WHERE Animals.project_id = 'MrsDash'
    AND label = 'sensor_data_temperature'
""").df()


display(df)

## Exporting Data to EDF
When it's easier to work with EDF files, we can export the data to an EDF file. This is useful for working with the data in other software packages.

Calling `export_to_edf(output_dir)` on a `DiveData` object creates one output EDF file for each recording in the `DiveData` relation, saved to `output_dir` with filename `<recording_id>.edf`. 

*Note: it currently requires a lot of memory. Can be improved.*<br/>
*Note: it's lacking support for most info fields in the EDF file.*

##### Example:

In [ ]:
import os
import importlib
import DiveDB.services.duck_pond
import DiveDB.services.dive_data
importlib.reload(DiveDB.services.duck_pond)
importlib.reload(DiveDB.services.dive_data)

from DiveDB.services.duck_pond import DuckPond

duckpond = DuckPond(os.environ["CONTAINER_DELTA_LAKE_PATH"])

dive_data = duckpond.get_delta_data(    
    animal_ids="apfo-001a",
    labels=["sensor_data_temperature", "derived_data_depth"],
    limit=1000000,
)

output_edf_paths = dive_data.export_to_edf(".tmp/my_output_dir/")
display(output_edf_paths)



In [ ]:

# What is stored in that EDF? Let's take a look!
from edfio import read_edf

edf_path = output_edf_paths[0] # There may be multiple, if multiple recordings were represented
edf = read_edf(edf_path)
print("EDF: ", edf)
print("Recording start date and time: ", edf.startdate, edf.starttime)
print("Recording: ", edf.recording)
print("Signals: ", edf.signals)
print("Patient: ", edf.patient)

# Investigating a single signal:
print("Signal data: ", edf.signals[0], edf.signals[0].data)
display(edf.signals[0].__dict__)

# And let's get back the metadata that we've stored:
import json
json.loads(edf.annotations[0].text)

## Importing exported EDF as an MNE Signal Array
For working with the data in MNE, we can export the data to an EDF and then import it to MNE.


##### Example:

In [ ]:
import importlib
import os
import DiveDB.services.duck_pond
importlib.reload(DiveDB.services.duck_pond)
from DiveDB.services.duck_pond import DuckPond

import mne

duckpond = DuckPond(os.environ["LOCAL_DELTA_PATH"])

conn = duckpond.get_delta_data(    
    animal_ids="mian-003",
    labels="ECG_ICA2",
    limit=1000000,
)

output_edf_paths = dive_data.export_to_edf(".tmp/my_output_dir/")
raw = mne.io.read_raw_edf(output_edf_paths[0])
display(raw)